In [ ]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.applications import Xception
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from collections import defaultdict

#GPU configuration
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print("GPU detected and configured.")
else:
    print("No GPU detected.")

dataset_dir = '/content/drive/MyDrive/Colab Notebooks/deep learning/breast cancer/cancer dataset'
img_size = (224, 224)
limit_per_class = 5000

data = []
labels = []
class_counts = defaultdict(int)

for class_name in sorted(os.listdir(dataset_dir)):
    class_path = os.path.join(dataset_dir, class_name)
    if os.path.isdir(class_path):
        images = sorted(os.listdir(class_path))[:limit_per_class]
        for img_name in images:
            img_path = os.path.join(class_path, img_name)
            try:
                img = load_img(img_path, target_size=img_size)
                img_array = img_to_array(img) / 255.0
                data.append(img_array)
                labels.append(class_name)
                class_counts[class_name] += 1
            except Exception as e:
                print(f"Skipping {img_path}: {e}")

print(f"Images loaded per class: {dict(class_counts)}")

#arrays
data = np.array(data, dtype='float32')
labels = np.array(labels)

le = LabelEncoder()
labels_encoded = le.fit_transform(labels)
num_classes = len(le.classes_)
labels = to_categorical(labels_encoded, num_classes=num_classes)

#split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.3, random_state=42)

# Load Xception base model
base_model = Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

# Model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    Dropout(0.7),
    Dense(num_classes, activation='softmax', kernel_regularizer=tf.keras.regularizers.l2(0.001))
])

# Compile model
model.compile(
    optimizer=Adam(learning_rate=0.0005),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train model
print("Training model...")
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=64,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping],
    verbose=1
)

# Evaluate
score = model.evaluate(X_test, y_test, verbose=0)
print(f'\nTest Accuracy: {score[1]*100:.2f}%')
print(f'Final Training Accuracy: {history.history["accuracy"][-1]*100:.2f}%')
print(f'Final Validation Accuracy: {history.history["val_accuracy"][-1]*100:.2f}%')


GPU detected and configured.
Images loaded per class: {'0_NoCancer': 1180, '1_Cancer': 1178}
83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
Training model...
Epoch 1/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 51s 1s/step - accuracy: 0.6318 - loss: 1.9609 - val_accuracy: 0.7825 - val_loss: 1.5727
Epoch 2/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 47s 444ms/step - accuracy: 0.8083 - loss: 1.4895 - val_accuracy: 0.8008 - val_loss: 1.3686
Epoch 3/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 18s 354ms/step - accuracy: 0.8358 - loss: 1.2795 - val_accuracy: 0.7839 - val_loss: 1.2309
Epoch 4/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 10s 361ms/step - accuracy: 0.8479 - loss: 1.1147 - val_accuracy: 0.7952 - val_loss: 1.1394
Epoch 5/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 10s 371ms/step - accuracy: 0.8284 - loss: 1.0386 - val_accuracy: 0.7938 - val_loss: 1.0597
Epoch 6/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 10s 377ms/step - accuracy: 0.8716 - loss: 0.9203 - val_accuracy: 0.7994 - val_loss: 1.0005
Epoch 7/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 12s 472ms/step - accuracy: 0.